In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import fu
from FRVRS import nu
from pandas import DataFrame
import os
import os.path as osp

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape) # (832366, 109)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(829116, 114)



# Gaze and Intent

Is there a correlation between gaze and intent, i.e., if a learner sees bleeding, do they treat it?

In [4]:

mask_series = (frvrs_logs_df.action_type == 'PLAYER_GAZE')
df = frvrs_logs_df[mask_series]
df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T

,821839,824931,813643,826652
action_type,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE,PLAYER_GAZE
action_tick,133165,198011,137962,69228
event_time,2023-09-11 13:05:00,2023-09-11 13:48:23,2023-09-07 12:45:29,2023-09-11 13:46:14
session_uuid,d0f826d3-a6db-4a6a-9ceb-5e17e3ff3d17,d9d58da9-9bdc-41ea-90fe-4c84db4635d9,384462fc-969c-42cd-944f-726634faba4f,d9d58da9-9bdc-41ea-90fe-4c84db4635d9
file_name,v.1.3/d0f826d3-a6db-4a6a-9ceb-5e17e3ff3d17.csv,v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv,v.1.3/384462fc-969c-42cd-944f-726634faba4f.csv,v.1.3/d9d58da9-9bdc-41ea-90fe-4c84db4635d9.csv
logger_version,1.3,1.3,1.3,1.3
scene_id,0,0,0,0
player_gaze_location,"(2.9, 0.0, 2.5)","(0.0, 0.0, 30.0)","(2.9, 0.0, 2.5)","(0.0, 0.0, 2.0)"
player_gaze_patient_id,Tutorial Military Marine Root,Lily_15 Root,Military Mike Jungle Combat_2_0 Root,Mike_3 Root
player_gaze_distance_to_patient,1.822125,1.014863,1.966808,2.119051


In [5]:

# Iterate over the patient ID and get all the bleeding
if nu.pickle_exists('bleeding_treated_df'): bleeding_treated_df = nu.load_object('bleeding_treated_df')
else:
    rows_list = []
    for (session_uuid, scene_id, patient_id), patient_df in frvrs_logs_df.sort_values(['action_tick']).groupby(fu.patient_groupby_columns):
        
        # Add the logger version and the groupby columns to the row dictionary
        row_dict = {}
        row_dict['logger_version'] = fu.get_logger_version(patient_df)
        for cn in fu.patient_groupby_columns: row_dict[cn] = eval(cn)
        
        # Did the responder gaze at this patient at least once?
        row_dict['gazed_at_patient'] = fu.is_patient_gazed_at(patient_df)
        
        # Is this patient bleeding?
        row_dict['is_patient_bleeding'] = fu.is_injury_hemorrhage(patient_df)
        
        # Was the bleeding treated?
        row_dict['bleeding_treated'] = fu.is_bleeding_correctly_treated(patient_df)
        
        # Add the row dictionary to the list
        rows_list.append(row_dict)
    
    # Create a data frame from the list of row dictionaries
    bleeding_treated_df = DataFrame(rows_list)
    nu.store_objects(bleeding_treated_df=bleeding_treated_df)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/bleeding_treated_df.pkl


In [6]:

# Display the minority combinations
groupby_columns = ['gazed_at_patient', 'is_patient_bleeding', 'bleeding_treated']
df = nu.get_minority_combinations(bleeding_treated_df, groupby_columns)
display(df.T)

,9757,6392,3565,2958
logger_version,1.3,1.3,1.3,1.0
session_uuid,fde2c201-36fc-4123-bf42-f1bff1b24969,a3d6d913-7755-4e8d-a174-d5e491c4eac7,5d916938-e0d3-47ee-8f55-12bf12283ee1,4727e60f-7e08-4595-b565-b6e80b7a8d87
scene_id,1,1,2,1
patient_id,Military Mike Jungle Combat_1_5 Root,Tutorial Military Marine Root,Lily_4 Root,Mike_5 Root
gazed_at_patient,True,True,False,False
is_patient_bleeding,False,True,True,True
bleeding_treated,False,True,False,True


In [7]:

groupby_columns = ['gazed_at_patient', 'is_patient_bleeding', 'bleeding_treated']
srs = bleeding_treated_df.groupby(groupby_columns).size()

# Get the count of cases where all conditions are met
count_true_true_true = srs.get((True, True, True), 0)

# Get the count of cases where gazed at patient is True and is patient bleeding is True
count_true_true = srs.get((True, True, False), 0) + srs.get((True, True, True), 0)

# Calculate the percentage
percentage = (count_true_true_true / count_true_true) * 100 if count_true_true > 0 else 0
if percentage == 100:
    yesno = 'Yes'
    percentage = f'all {count_true_true_true}'
elif percentage < 100:
    yesno = 'Not quite'
    percentage = f'{percentage:.1f}%'
elif percentage == 0:
    yesno = 'No'
    percentage = 'none'

print(f'{yesno}, {percentage} of the responders who gazed at a patient and discovered they were bleeding, treated the patient.')

Yes, all 20 of the responders who gazed at a patient and discovered they were bleeding, treated the patient.


In [11]:

srs.to_frame().rename(columns={0: 'task_count'}).reset_index(drop=False)

,gazed_at_patient,is_patient_bleeding,bleeding_treated,task_count
0,False,False,False,7881
1,False,True,False,25
2,False,True,True,1932
3,True,False,False,16
4,True,True,True,20
